In [18]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [3]:
class myCallback(# your code
    tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('acc')>0.995):
        print("\nReached 99.5% accuracy so cancelling training!")
        self.model.stop_training = True

In [4]:
callbacks = myCallback()

In [5]:
model = tf.keras.models.Sequential([
    # Your Code Here
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=(640,480,3)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(128, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(5 , activation = tf.nn.softmax)
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 636, 476, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 212, 158, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 210, 156, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 70, 52, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 68, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 34, 25, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 108800)            0

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(rescale=1.0/255)

In [19]:
train_csv = pd.read_csv('TrainAnnotations.csv')

In [20]:
filenames = ['TrainData/' + fname for fname in train_csv['file_name'].tolist()]

In [21]:
labels = train_csv['annotation'].tolist()

In [40]:
Image_SIZE_H = 640
Image_SIZE_W = 480

def _parse_fn(filename, label):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
  image_resized = tf.image.resize(image_normalized, (Image_SIZE_H, Image_SIZE_W))
  return image_resized, label

In [41]:
train_filenames, val_filenames, train_labels, val_labels = train_test_split(filenames,
                 labels,
                 train_size=0.8,
                 random_state=42)

In [42]:
BATCH_SIZE = 32

train_data = tf.data.Dataset.from_tensor_slices((tf.constant(train_filenames), tf.constant(train_labels))).map(_parse_fn).shuffle(buffer_size=10000).batch(BATCH_SIZE)


val_data = tf.data.Dataset.from_tensor_slices((tf.constant(val_filenames), tf.constant(val_labels))).map(_parse_fn).batch(BATCH_SIZE)

In [43]:
train_data

<BatchDataset shapes: ((None, 640, 480, None), (None,)), types: (tf.float32, tf.int32)>

In [45]:
val_data

<BatchDataset shapes: ((None, 640, 480, None), (None,)), types: (tf.float32, tf.int32)>

In [28]:
labels

[0,
 0,
 0,
 4,
 0,
 3,
 4,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 3,
 1,
 4,
 3,
 0,
 4,
 1,
 3,
 1,
 0,
 2,
 1,
 3,
 3,
 0,
 2,
 4,
 2,
 2,
 3,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 3,
 1,
 3,
 0,
 2,
 0,
 2,
 2,
 3,
 3,
 0,
 3,
 0,
 1,
 0,
 0,
 1,
 1,
 3,
 0,
 4,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 4,
 0,
 4,
 3,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 4,
 3,
 4,
 4,
 0,
 1,
 0,
 3,
 3,
 3,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 3,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 3,
 0,
 0,
 1,
 0,
 2,
 1,
 1,
 4,
 0,
 2,
 0,
 0,
 4,
 0,
 3,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 1,
 3,
 1,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 1,
 1,
 0,
 1,
 3,
 0,
 1,
 0,
 0,
 2,
 4,
 1,
 0,
 4,
 1,
 0,
 4,
 2,
 0,
 0,
 1,
 4,
 2,
 1,
 0,
 1,
 0,
 0,
 2,
 4,
 1,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 3,
 0,
 4,
 1,
 0,
 1,
 0,
 0,
 0,
 3,
 1,
 3,
 2,
 3,
 3,
 0,
 0,
 0,
 0,
 1,
 3,
 0,
 3,
 0,
 4,
 3,
 2,
 0,
 0,
 2,
 0,
 0,
 3,
 0,
 2,
 1,
 0,
 1,
 3,
 0,
 4,
 0,
 0,
 3,


In [46]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam',metrics=['acc'])

In [47]:
y = LabelEncoder().fit_transform(labels)

In [48]:
y

array([0, 0, 0, ..., 0, 2, 1])

In [49]:
len(y)

1025

In [50]:
len(labels)

1025

In [51]:
model.fit(train_data, epochs=25, validation_data = val_data, verbose=0)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2869, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-51-6765a1324c55>", line 1, in <module>
    model.fit(train_data, epochs=25, validation_data = val_data, verbose=0)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2_utils.py", line 98, in execution_function
    distributed_function(input_fn))
 

KeyboardInterrupt: 